In [2]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

import cv2

/home/clarence/.local/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/clarence/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to

In [3]:
data = DataLoader.from_folder(r"./datasets/tensorflow/photos/")
train_data, test_data = data.split(0.9)

INFO:tensorflow:Load image with size: 2917, num_label: 2, labels: Moving, still.


In [4]:
model = image_classifier.create(train_data)

INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2 (HubKe  (None, 1280)             3413024   
 rasLayerV1V2)                                                   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 3,415,586
Trainable params: 2,562
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/5


2023-05-26 11:02:09.508833: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


82/82 [==============================] - 5s 29ms/step - loss: 0.4107 - accuracy: 0.8628
Epoch 2/5
82/82 [==============================] - 3s 30ms/step - loss: 0.3015 - accuracy: 0.9447
Epoch 3/5
82/82 [==============================] - 2s 29ms/step - loss: 0.2837 - accuracy: 0.9562
Epoch 4/5
82/82 [==============================] - 2s 29ms/step - loss: 0.2757 - accuracy: 0.9607
Epoch 5/5
82/82 [==============================] - 2s 29ms/step - loss: 0.2710 - accuracy: 0.9638


In [5]:
loss, accuracy = model.evaluate(test_data)

10/10 [==============================] - 1s 45ms/step - loss: 0.2667 - accuracy: 0.9658


In [6]:
model.export(export_dir='.')

2023-05-26 11:02:24.103126: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmp5h76o50m/assets


INFO:tensorflow:Assets written to: /tmp/tmp5h76o50m/assets
/home/clarence/.local/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-26 11:02:27.758516: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-05-26 11:02:27.758545: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


fully_quantize: 0, inference_type: 6, input_inference_type: 3, output_inference_type: 3
INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmpihu_ll7_/labels.txt


INFO:tensorflow:Saving labels in /tmp/tmpihu_ll7_/labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


In [20]:
video_path = "./datasets/vid2.mp4"
output_path = "resized_video.mp4"
target_width = 224
target_height = 224

# resize_video(video_path, output_path, target_width, target_height)

# Load the video using OpenCV
cap = cv2.VideoCapture(output_path)

# Initialize an empty list to store the frame classifications
frame_classifications = []

# Loop through the frames of the video
while(True):
    ret, frame = cap.read() # Read the next frame
    if not ret: # Break the loop if the video has ended
        break

    ''' Formulate Input Data (frame_rgb) '''
    # Convert the frame to RGB format
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Make frame input data and ensure its type matches the model
    frame_rgb = np.expand_dims(frame_rgb, axis=0)

    ''' Classify the Frame '''
    interpreter = tf.lite.Interpreter(model_path="model.tflite")
    interpreter.allocate_tensors()

    # get_output_details() and get_input_details() return list of dictionaries of tensor details
    # keys: name, index, shape, shape_signature, dtype, quantization, ...
    # len(input) = len(output) = 1, so access the first element
    output = interpreter.get_output_details()
    input = interpreter.get_input_details()
    output_index = output[0]['index']
    input_index = input[0]['index']

    # set input -> invoke -> access output
    interpreter.set_tensor(input_index, frame_rgb)
    interpreter.invoke()


    
    output_data = interpreter.get_tensor(output[0]['index']) ### potentially wrong ###
    # If the output_data shape is (batch_size, num_classes), select the first frame
    output_data = output_data[0]

    # Find the index of the highest probability
    predicted_index = np.argmax(output_data)

    # Assuming you have a list of class labels corresponding to the model's output classes
    class_labels = ["Moving", "Still"]

    # Get the predicted class label
    predicted_class = class_labels[predicted_index]

    # Print the predicted class label
    print("Predicted Class:", predicted_class)
    frame_classifications.append(predicted_class)
    '''
    print("output: ")
    print(output_data)
    '''
    '''Append Classification Result to frame_classifications'''
    '''
    # Add the frame classification to the list
    frame_classifications.append(label)

    # Display the frame
    cv2.imshow("Video", frame)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    '''
    #frame_classifications.append(output_data)

# Release the video capture and close the OpenCV windows (Do we need this?)
# cap.release()
# cv2.destroyAllWindows()

''' SHOW CLASSIFICATION RESULTS '''
# Convert the frame classifications to a numpy array
#frame_classifications = np.array(predicted_class)
# Print the frame classifications
print("Frame Classifications:", frame_classifications)

Predicted Class: Still
Predicted Class: Still
Predicted Class: Still
Predicted Class: Still
Predicted Class: Still
Predicted Class: Still
Predicted Class: Still
Predicted Class: Still
Predicted Class: Still
Predicted Class: Still


KeyboardInterrupt: 